<a href="https://colab.research.google.com/github/kwanglo/mge51101-20195171/blob/master/final_project/combined_cnn_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [116]:
!nvidia-smi

Sun Jun  7 12:42:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    31W / 250W |    995MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip3 install konlpy
!pip3 install soynlp

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.6MB 56.1MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 419kB 4.6MB/s 


In [0]:
import os
import re

from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Twitter

import pandas as pd
import numpy as np

In [0]:
path='/gdrive/My Drive/Colab Notebooks/Final Project/dataset/'
data_emo = pd.read_excel(path+"한국어_단발성_대화_데이터셋.xlsx")
data_utter = pd.read_csv(path+"fci_data.csv")
train_utter = pd.read_csv(path+"fci_train.csv")
test_utter = pd.read_csv(path+"fci_test.csv")

# Prepare dataset

In [0]:
data_emo = data_emo.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4','공포',5468])

In [6]:
data_emo.groupby("Emotion").count()

,Sentence
Emotion,
공포,5468
놀람,5898
분노,5665
슬픔,5267
중립,4830
행복,6037
혐오,5429


In [0]:
# Char to int
data_emo.loc[(data_emo.Emotion == "중립"),"Emotion"] = 0
data_emo.loc[(data_emo.Emotion == "공포"),"Emotion"] = 1
data_emo.loc[(data_emo.Emotion == "놀람"),"Emotion"] = 2
data_emo.loc[(data_emo.Emotion == "분노"),"Emotion"] = 3
data_emo.loc[(data_emo.Emotion == "슬픔"),"Emotion"] = 4
data_emo.loc[(data_emo.Emotion == "행복"),"Emotion"] = 5
data_emo.loc[(data_emo.Emotion == "혐오"),"Emotion"] = 6

In [0]:
# 숫자로 이미지 나눌때 
#중립
Neutral = data_emo[data_emo["Emotion"] == 0]
#공포
Fear = data_emo[data_emo["Emotion"] == 1]
Fear.head()
#놀람
Surprise = data_emo[data_emo["Emotion"] == 2]
#분노
Anger = data_emo[data_emo["Emotion"] == 3]
#슬픔
Sad = data_emo[data_emo["Emotion"] == 4]
#행복
Happy = data_emo[data_emo["Emotion"] == 5]
#혐오
Disgust = data_emo[data_emo["Emotion"] == 6]

In [0]:
rnd_num = 2020
Fear_train = Fear.sample(frac=0.7, random_state=rnd_num)
Fear_test = Fear.drop(Fear_train.index)

Surprise_train = Surprise.sample(frac=0.7, random_state=rnd_num)
Surprise_test = Surprise.drop(Surprise_train.index)

Anger_train = Anger.sample(frac=0.7, random_state=rnd_num)
Anger_test = Anger.drop(Anger_train.index)

Sad_train = Sad.sample(frac=0.7, random_state=rnd_num)
Sad_test = Sad.drop(Sad_train.index)

Neutral_train = Neutral.sample(frac=0.7, random_state=rnd_num)
Neutral_test = Neutral.drop(Neutral_train.index)

Happy_train = Happy.sample(frac=0.7, random_state=rnd_num)
Happy_test = Happy.drop(Happy_train.index)

Disgust_train = Disgust.sample(frac=0.7, random_state=rnd_num)
Disgust_test = Disgust.drop(Disgust_train.index)

train_emo = pd.concat([Fear_train,Surprise_train,Anger_train,Sad_train,Neutral_train,Happy_train,Disgust_train])
test_emo = pd.concat([Fear_test,Surprise_test,Anger_test,Sad_test,Neutral_test,Happy_test,Disgust_test])

In [10]:
data_utter.groupby("label").count()
#Fragments(FR) - 0
#Statements(S) - 1
#Questions(Q) - 2
#Commands(C) - 3
#Rhetorical questions(RQ) - 4
#Rhetorical commands(RC) - 5
#FragmeIntonation-dependent utterances(IU) - 6

,text
label,
0,6009
1,18300
2,17869
3,12968
4,1745
5,1087
6,3277


# Data preprocessing - Emotion

In [0]:
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def tokenizer(text): # create a tokenizer function
    okt = Okt()
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    x = okt.morphs(text , stem= True)
    return x

In [0]:
stop_words_set = pd.read_csv(path+'stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words)

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from soynlp.tokenizer import MaxScoreTokenizer
SEED = 3432

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT_emo = data.Field(tokenize=tokenizer, stop_words = stop_words)
LABEL_emo = data.LabelField()

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [0]:
from torchtext.data import TabularDataset
fields_emo = [("Sentence", TEXT_emo),("Emotion", LABEL_emo)]

In [0]:
train_emo,valid_emo, test_emo = data.TabularDataset.splits(
                                        path = path,
                                        train = 'sentiment_train.csv',
                                        validation = 'sentiment_valid.csv',
                                        test = 'sentiment_test.csv',
                                        format = 'csv',
                                        fields = fields_emo,
                                        skip_header = True
)

In [55]:
vars(train_emo[3])

{'Emotion': '5', 'Sentence': ['어제', '런닝맨', '완전', '재밌다']}

In [0]:
import torchtext
vec = torchtext.vocab.Vectors('wiki.ko.vec', cache=path)

In [0]:
MAX_VOCAB_SIZE = 25000

TEXT_emo.build_vocab(train_emo, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = vec, 
                 unk_init = torch.Tensor.normal_)

LABEL_emo.build_vocab(train_emo)

In [0]:
from torchtext.data import Iterator, BucketIterator
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator_emo, valid_iterator_emo, test_iterator_emo = data.BucketIterator.splits(
    (train_emo, valid_emo, test_emo), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)


# Data preprocessing - Utterance

In [0]:
SEED = 3432

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT_utt = data.Field(tokenize=tokenizer, stop_words = stop_words)
LABEL_utt = data.LabelField()

In [0]:
fields_utt = [("text", TEXT_utt),("label", LABEL_utt)]

In [0]:
train_utt,valid_utt, test_utt = data.TabularDataset.splits(
                                        path = path,
                                        train = 'utterance_train.csv',
                                        validation = 'utterance_valid.csv',
                                        test = 'utterance_test.csv',
                                        format = 'csv',
                                        fields = fields_utt,
                                        skip_header = True
)

In [0]:
MAX_VOCAB_SIZE = 25000

TEXT_utt.build_vocab(train_utt, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = vec, 
                 unk_init = torch.Tensor.normal_)

LABEL_utt.build_vocab(train_utt)

In [0]:
train_iterator_utt, valid_iterator_utt, test_iterator_utt = data.BucketIterator.splits(
    (train_utt, valid_utt, test_utt), 
    batch_size = BATCH_SIZE, 
    device = device, sort = False)

# Model building

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_emo(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)        
        embedded = self.embedding(text)

        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [0]:
class CNN_utt(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        text = text.permute(1, 0)        
        embedded = self.embedding(text)

        embedded = embedded.unsqueeze(1)
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cat = self.dropout(torch.cat(pooled, dim = 1))
            
        return self.fc(cat)

In [0]:
INPUT_DIM_emo = len(TEXT_emo.vocab)
OUTPUT_DIM_emo = len(LABEL_emo.vocab)
PAD_IDX_emo = TEXT_emo.vocab.stoi[TEXT_emo.pad_token]

INPUT_DIM_utt = len(TEXT_utt.vocab)
OUTPUT_DIM_utt = len(LABEL_utt.vocab)
PAD_IDX_utt = TEXT_utt.vocab.stoi[TEXT_utt.pad_token]

EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
DROPOUT = 0.5

model_emo = CNN_emo(INPUT_DIM_emo, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM_emo, DROPOUT, PAD_IDX_emo)
model_utt = CNN_utt(INPUT_DIM_utt, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM_utt, DROPOUT, PAD_IDX_utt)

In [91]:
def count_parameters_emo(model_emo):
    return sum(p.numel() for p in model_emo.parameters() if p.requires_grad)

print(f'The model has {count_parameters_emo(model_emo):,} trainable parameters')

The model has 5,058,307 trainable parameters


In [92]:
def count_parameters_utt(model_utt):
    return sum(p.numel() for p in model_utt.parameters() if p.requires_grad)

print(f'The model has {count_parameters_utt(model_utt):,} trainable parameters')

The model has 3,218,707 trainable parameters


In [0]:
pretrained_embeddings_emo = TEXT_emo.vocab.vectors
pretrained_embeddings_utt = TEXT_utt.vocab.vectors

In [0]:
UNK_IDX_emo = TEXT_emo.vocab.stoi[TEXT_emo.unk_token]
UNK_IDX_utt = TEXT_utt.vocab.stoi[TEXT_utt.unk_token]

model_emo.embedding.weight.data[UNK_IDX_emo] = torch.zeros(EMBEDDING_DIM)
model_emo.embedding.weight.data[PAD_IDX_emo] = torch.zeros(EMBEDDING_DIM)

model_utt.embedding.weight.data[UNK_IDX_utt] = torch.zeros(EMBEDDING_DIM)
model_utt.embedding.weight.data[PAD_IDX_utt] = torch.zeros(EMBEDDING_DIM)

In [0]:
import torch.optim as optim

optimizer_emo = optim.Adam(model_emo.parameters())
optimizer_utt = optim.Adam(model_utt.parameters())
criterion = nn.CrossEntropyLoss()

model_emo = model_emo.to(device)
model_utt = model_utt.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train_model_emo(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.Sentence)
        
        loss = criterion(predictions, batch.Emotion)
        
        acc = categorical_accuracy(predictions, batch.Emotion)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_model_emo(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Sentence)
            
            loss = criterion(predictions, batch.Emotion)
            
            acc = categorical_accuracy(predictions, batch.Emotion)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def train_model_utt(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)        
        loss = criterion(predictions, batch.label)        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_model_utt(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)            
            loss = criterion(predictions, batch.label)            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Model training

In [102]:
#Emotion
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss_emo, train_acc_emo = train_model_emo(model_emo, train_iterator_emo, optimizer_emo, criterion)
    valid_loss_emo, valid_acc_emo = evaluate_model_emo(model_emo, valid_iterator_emo, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss_emo < best_valid_loss:
        best_valid_loss = valid_loss_emo
        torch.save(model_emo.state_dict(), 'emo-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss_emo:.3f} | Train Acc: {train_acc_emo*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss_emo:.3f} |  Val. Acc: {valid_acc_emo*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 1.798 | Train Acc: 29.44%
	 Val. Loss: 1.557 |  Val. Acc: 39.74%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 1.488 | Train Acc: 43.36%
	 Val. Loss: 1.522 |  Val. Acc: 40.50%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 1.290 | Train Acc: 51.94%
	 Val. Loss: 1.562 |  Val. Acc: 40.91%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 1.100 | Train Acc: 59.39%
	 Val. Loss: 1.595 |  Val. Acc: 41.77%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.902 | Train Acc: 67.36%
	 Val. Loss: 1.697 |  Val. Acc: 41.57%
Epoch: 06 | Epoch Time: 0m 10s
	Train Loss: 0.739 | Train Acc: 73.72%
	 Val. Loss: 1.834 |  Val. Acc: 41.50%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.602 | Train Acc: 79.13%
	 Val. Loss: 1.994 |  Val. Acc: 41.61%
Epoch: 08 | Epoch Time: 0m 10s
	Train Loss: 0.504 | Train Acc: 82.13%
	 Val. Loss: 2.163 |  Val. Acc: 41.61%
Epoch: 09 | Epoch Time: 0m 10s
	Train Loss: 0.419 | Train Acc: 85.76%
	 Val. Loss: 2.351 |  Val. Acc: 41.60%
Epoch: 10 | Epoch T

In [103]:
model_emo.load_state_dict(torch.load('emo-model.pt'))

test_loss_emo, test_acc_emo = evaluate_model_emo(model_emo, test_iterator_emo, criterion)

print(f'Test Loss: {test_loss_emo:.3f} | Test Acc: {test_acc_emo*100:.2f}%')

Test Loss: 1.530 | Test Acc: 40.69%


In [105]:
#Utterance
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss_utt, train_acc_utt = train_model_utt(model_utt, train_iterator_utt, optimizer_utt, criterion)
    valid_loss_utt, valid_acc_utt = evaluate_model_utt(model_utt, valid_iterator_utt, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss_utt < best_valid_loss:
        best_valid_loss = valid_loss_utt
        torch.save(model_utt.state_dict(), 'utt-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss_utt:.3f} | Train Acc: {train_acc_utt*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss_utt:.3f} |  Val. Acc: {valid_acc_utt*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 9s
	Train Loss: 0.715 | Train Acc: 73.93%
	 Val. Loss: 1.018 |  Val. Acc: 66.53%
Epoch: 02 | Epoch Time: 0m 9s
	Train Loss: 0.612 | Train Acc: 77.51%
	 Val. Loss: 1.081 |  Val. Acc: 66.52%
Epoch: 03 | Epoch Time: 0m 9s
	Train Loss: 0.534 | Train Acc: 80.37%
	 Val. Loss: 1.142 |  Val. Acc: 66.39%
Epoch: 04 | Epoch Time: 0m 9s
	Train Loss: 0.475 | Train Acc: 82.45%
	 Val. Loss: 1.224 |  Val. Acc: 67.41%
Epoch: 05 | Epoch Time: 0m 9s
	Train Loss: 0.430 | Train Acc: 84.22%
	 Val. Loss: 1.338 |  Val. Acc: 65.76%
Epoch: 06 | Epoch Time: 0m 9s
	Train Loss: 0.384 | Train Acc: 85.69%
	 Val. Loss: 1.418 |  Val. Acc: 66.57%
Epoch: 07 | Epoch Time: 0m 9s
	Train Loss: 0.358 | Train Acc: 86.95%
	 Val. Loss: 1.518 |  Val. Acc: 67.62%
Epoch: 08 | Epoch Time: 0m 9s
	Train Loss: 0.332 | Train Acc: 87.66%
	 Val. Loss: 1.602 |  Val. Acc: 66.28%
Epoch: 09 | Epoch Time: 0m 9s
	Train Loss: 0.315 | Train Acc: 88.50%
	 Val. Loss: 1.699 |  Val. Acc: 66.48%
Epoch: 10 | Epoch Time: 0m 9

In [107]:
model_utt.load_state_dict(torch.load('utt-model.pt'))

test_loss_utt, test_acc_utt = evaluate_model_utt(model_utt, test_iterator_utt, criterion)

print(f'Test Loss: {test_loss_utt:.3f} | Test Acc: {test_acc_utt*100:.2f}%')

Test Loss: 1.032 | Test Acc: 66.44%


In [0]:
def predict_emo(model, sentence, min_len = 4):
    model.eval()
    # 이 부분에서 그냥 바로 tokenizing
    tokenized = tokenizer(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT_emo.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [0]:
def emotion_classifier(logits):
  global sentiment
  if logits == "0":
    sentiment = '중립'
  elif logits == "1":
    sentiment = '공포'
  elif logits == "2":
    sentiment = '놀람'
  elif logits == "3":
    sentiment = '분노'
  elif logits == "4":
    sentiment = '슬픔'
  elif logits == "5":
    sentiment = '행복'
  elif logits == "6":
    sentiment = '혐오'

  return sentiment

In [0]:
def predict_utt(model, sentence, min_len = 4):
    model.eval()
    # 이 부분에서 그냥 바로 tokenizing
    tokenized = tokenizer(sentence)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT_utt.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [0]:
def utterance_classifier(logits):
  global utterance
  if logits == "0":
    utterance = '미완'
  elif logits == "1":
    utterance = '서술'
  elif logits == "2":
    utterance = '질문'
  elif logits == "3":
    utterance = '요구'
  elif logits == "4":
    utterance = '수사의문'
  elif logits == "5":
    utterance = '수사명령'
  elif logits == "6":
    utterance = '억양'

  return utterance

In [124]:
pred_sentence = input()
pred_emo = predict_emo(model_emo, pred_sentence)
pred_utt = predict_utt(model_utt, pred_sentence)
logit_1 = LABEL_emo.vocab.itos[pred_emo]
logit_2 = LABEL_utt.vocab.itos[pred_utt]

print(f'이 문장의 감정은 {emotion_classifier(logit_1)}이고, 발화 의도는 {utterance_classifier(logit_2)}입니다')

프로젝트가 마무리되고 있어서 너무 좋다.
이 문장의 감정은 행복이고, 발화 의도는 서술입니다


In [0]:
## 이제 Class별로 정확도 구분할 수 있게